## ANN Regressor

Un fia de roba

In [124]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import pandas as pd

X_train = pd.read_csv('x_train_preprocessed_minmax.csv')
X_test = pd.read_csv('x_test_preprocessed_minmax.csv')
y_train = pd.read_csv('y_train_preprocessed_minmax.csv')
y_test = pd.read_csv('y_test_preprocessed_minmax.csv')

y_train = y_train.to_numpy().flatten() #y_train flattened, works better

oh_neighbor = []
for col in X_train.columns:
    if 'Neighborhood_b' in col:
        oh_neighbor.append(col)

X_train.drop(columns=oh_neighbor, inplace=True)
X_test.drop(columns=oh_neighbor, inplace=True)

porch = ['Wood_Deck_SF', 'Open_Porch_SF', 'Enclosed_Porch', 'Three_season_porch', 'Screen_Porch']
surface = ['Total_Finished_Bsmt_SF', 'First_Flr_SF', 'Second_Flr_SF', 'Garage_Area']
baths = ['Full_Bath', 'Half_Bath', 'Bsmt_Full_Bath', 'Bsmt_Half_Bath']

X_train.drop(columns=porch, inplace=True)
X_test.drop(columns=porch, inplace=True)

X_train.drop(columns=surface, inplace=True)
X_test.drop(columns=surface, inplace=True)

X_train.drop(columns=baths, inplace=True)
X_test.drop(columns=baths, inplace=True)

All the feature which have been encoded are removed, this solution significatively reduces the loss parameter.

## Network initialization and setup

Using single hidden layer with half of the units of the input one.
The output layer has a single units due to the necessity to predict only the Sale Price.

For the input and hidden layers the activation function chosen is relu because of its good speed of train and simplicity to compute.
The output has instead the linear activation function for predicting the float value.

The kernel initializer is always normal so it use a normal distribution to generate tensors.

In [125]:
model = models.Sequential()

model.add(layers.Dense(units=189, kernel_initializer='normal', activation='linear', input_shape=[188, ]))
model.add(layers.Dense(units=95, kernel_initializer='normal', activation='elu'))
model.add(layers.Dense(units=47, kernel_initializer='normal', activation='linear'))
model.add(layers.Dense(units=17, kernel_initializer='normal', activation='elu'))
model.add(layers.Dense(units=1, kernel_initializer='normal', activation='relu'))

opt = optimizers.Adam(learning_rate=0.0015, beta_1=0.9, beta_2=0.999, amsgrad=False)

## Model evaluation

In [126]:
from tensorflow_addons.metrics import RSquare
model.compile(loss='mean_squared_logarithmic_error', optimizer=opt, metrics=[RSquare()])
model.fit(X_train, y_train,epochs=140)

loss = model.evaluate(X_test, y_test)
print('Test loss:', loss)


Epoch 101/140
69/69 [==============================] - 1s 1ms/step - loss: 76.4771 - r_square: -5.0893
Epoch 102/140
69/69 [==============================] - 0s 1ms/step - loss: 18.5661 - r_square: -4.9460
Epoch 103/140
69/69 [==============================] - 0s 1ms/step - loss: 8.7905 - r_square: -4.5784
Epoch 104/140
69/69 [==============================] - 0s 1ms/step - loss: 4.8040 - r_square: -4.0252
Epoch 105/140
69/69 [==============================] - 0s 1ms/step - loss: 2.7909 - r_square: -3.3676
Epoch 106/140
69/69 [==============================] - 0s 1ms/step - loss: 1.6801 - r_square: -2.6765
Epoch 107/140
69/69 [==============================] - 0s 1ms/step - loss: 1.0394 - r_square: -2.0274
Epoch 108/140
69/69 [==============================] - 0s 1ms/step - loss: 0.6605 - r_square: -1.4627
Epoch 109/140
69/69 [==============================] - 0s 1ms/step - loss: 0.4344 - r_square: -1.0026
Epoch 110/140
69/69 [==============================] - 0s 1ms/step - loss: 0.299

## Predicted values

In [127]:
from sklearn.metrics import r2_score

y_pred = model.predict(X_test)

print(r2_score(y_test, y_pred))

23/23 [==============================] - 0s 727us/step
0.5231729774229245
